# NeuralNets for b-hadron pT regression

In [ ]:
%load_ext autoreload
%autoreload 2
from bob import *
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from tqdm import tqdm

Load dataframe

In [ ]:
inputFileName = 'MC16d_newTrain_Zprime.pkl'

# Subsample the dataset for fast execution
subsampleFlag = False
gpuFlag = True

In [ ]:
tree = pd.read_pickle(inputFileName)
features = select_features(tree, to_remove=[])

# Add flag for missing values in SV1
tree['nan_flag'] = tree['jet_sv1_sig3d'] == -100
features.append('nan_flag')

tree['jet_bH_pt'] = tree['jet_bH_pt'].apply(lambda x: x[0])

if subsampleFlag:
    tree = tree.head(int(tree.shape[0]*0.05))
    num_boost_round=100
else:
    num_boost_round=1000
    
# Replace missing values with NaNs
d = dict.fromkeys([-100, -1, -99, -1000], np.nan)
tree.replace(d, inplace=True)

# Normalization
tree[features] = tree[features].apply(lambda x: (x-x.mean())/x.std(), axis=0)

tree.replace(np.nan, 0, inplace=True)

tree['jet_LabDr_HadF'].replace(to_replace=5, value=2, inplace=True) 
tree['jet_LabDr_HadF'].replace(to_replace=4, value=1, inplace=True) 

In [ ]:
bh_pt_std = tree['jet_bH_pt'].std()
pt_mean = tree['jet_pt'].mean()
pt_std = tree['jet_pt'].std()

In [ ]:
tree['jet_bH_pt'] = tree['jet_bH_pt'] / bh_pt_std
tree['jet_pt'] = (tree['jet_pt'] - pt_mean) / pt_std

In [ ]:
train, test = train_test_splitting(tree)
train = train.head(train.shape[0]//100*100)

In [ ]:
del tree

In [ ]:
train_input, train_target = Variable(torch.from_numpy(train[features].values.astype(np.float32))), \
                            Variable(torch.from_numpy((train['jet_bH_pt'].values.astype(np.float32))))

In [ ]:
test_input, test_target = Variable(torch.from_numpy(test[features].values.astype(np.float32)), volatile=True), \
                            Variable(torch.from_numpy((test['jet_bH_pt'].values.astype(np.float32))), volatile=True)

In [ ]:
if gpuFlag:
    train_input, train_target, test_input, test_target = train_input.cuda(), train_target.cuda(), test_input.cuda(), test_target.cuda()

# Define the network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(25, 100)
        self.fc1_bn = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 100)
        self.fc2_bn = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = self.fc1_bn(F.relu(self.fc1(x)))
        x = self.fc2_bn(F.relu(self.fc2(x)))
        x = self.fc3(x)
        x = torch.mul(x,x).flatten()
        return x
    
model, criterion = Net(), nn.MSELoss()

Training

In [ ]:
if gpuFlag:
    model.cuda()

learning_rate = .001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

mini_batch_size = 100
generalization_loss = []

for e in tqdm(range(30)):
    sum_loss = 0
    model.train()
    idxs = np.random.permutation(train_input.size(0))
    # We do this with mini-batches
    for b in range(0, train_input.size(0), mini_batch_size):
        output = model(train_input[idxs].narrow(0, b, mini_batch_size))
        loss = criterion(output, train_target[idxs].narrow(0, b, mini_batch_size))
        sum_loss = sum_loss + loss.item()
        optimizer.zero_grad()        
        loss.backward()
        optimizer.step()
    
    model.eval()
    output = model(test_input)
    loss = criterion(output, test_target)
    if loss.item() < 1e3:
        generalization_loss.append(loss.item())
    
    scheduler.step()
    
print(e, sum_loss)
plt.plot(generalization_loss);
plt.grid()
plt.show()

Save the model

In [ ]:
#torch.save(model.state_dict(), 'mymodel.pt')

Load the model

In [ ]:
#model.load_state_dict(torch.load('mymodel.pt'))

In [ ]:
if gpuFlag:
    output = output.cpu()
pt_pred = output.data.numpy()

In [ ]:
pt_pred = pt_pred * pt_std
test['jet_bH_pt'] = test['jet_bH_pt'] * bh_pt_std
test['jet_pt'] = (test['jet_pt'] * pt_std) + pt_mean


In [ ]:
plt.figure(figsize=(12,5.5))
plt.hist((pt_pred[test['jet_LabDr_HadF']==5],pt_pred[test['jet_LabDr_HadF']==4],pt_pred[test['jet_LabDr_HadF']==0], \
         test['jet_pt'][test['jet_LabDr_HadF']==5],test['jet_pt'][test['jet_LabDr_HadF']==4],pt_pred[test['jet_LabDr_HadF']==0]),\
         log=True, density=True, label=('b','c','l'), bins=120, histtype = 'step');
plt.grid()
plt.legend()
#plt.xlim([0,1.7e6])
plt.xlabel('$p_T$ predicted (regression) [TeV]')
plt.ylabel('prob. density')
plt.show()

In [ ]:
x = np.linspace(0,1.4,50)

plt.figure(1,) #figsize=(10,6))
degrees = [1]       # list of degrees of x to use
matrix = np.stack([test['jet_bH_pt']**d for d in degrees], axis=-1)   # stack them like columns
#slope, r, _, _ = np.linalg.lstsq(matrix, pt_pred)
slope = 1
plt.plot(x, x*slope, 'r')
print(slope, 1-sum((test['jet_bH_pt'] - pt_pred)**2)/sum((pt_pred - pt_pred.mean())**2) )

h = np.histogram2d(test['jet_bH_pt'], pt_pred, bins=(np.linspace(0,1.4e6,112),np.linspace(0,1.4e6,112)), density=True)
plt.imshow(h[0].T, norm=matplotlib.colors.LogNorm(), extent=[0,1.4,0,1.4], origin='lower')
plt.xlabel('jet_bH_pt [TeV]')
plt.ylabel('regression_pt [TeV]')
plt.colorbar()
plt.grid()

plt.figure(2,) #figsize=(10,6))

slope, r, _, _ = np.linalg.lstsq(matrix, test['jet_pt'])
plt.plot(x, x*slope, 'r')
print(slope, 1-r/sum((test['jet_pt'] - test['jet_pt'].mean())**2) )

h = np.histogram2d(test['jet_bH_pt'], test['jet_pt'], bins=(np.linspace(0,1.4e6,112),np.linspace(0,1.4e6,112)), density=True)
plt.imshow(h[0].T, norm=matplotlib.colors.LogNorm(), extent=[0,1.4,0,1.4], origin='lower')
plt.xlabel('jet_bH_pt [TeV]')
plt.ylabel('jet_pt [TeV]')
plt.colorbar()
plt.grid()
plt.savefig('jetbHpt_jetpt.png')